# AWA2

In [1]:
%matplotlib inline
import os,sys
pwd = os.getcwd()
parent = '/'.join(pwd.split('/')[:-3])
sys.path.insert(0,parent)
os.chdir(parent)

In [2]:
#%%
print('-'*30)
print(os.getcwd())
print('-'*30)
#%%
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F  
import pandas as pd
from core.DAZLE import DAZLE
from core.AWA2DataLoader import AWA2DataLoader
from core.helper_func import eval_zs_gzsl,visualize_attention,eval_zs_gzsl_k#,get_attribute_attention_stats
from global_setting import NFS_path_AoA,save_NFS
#from core.Scheduler import Scheduler
import importlib
import pdb
import numpy as np
import argparse
import time
from importlib import reload
import matplotlib.pyplot as plt

------------------------------
/home/project_amadeus/home/hbdat/[RELEASE]_CompositionalZSL
------------------------------


In [3]:
import core.helper_func
reload(core.helper_func)
from core.helper_func import next_unseen_batch,compute_mean_real_unseen,compute_mean_real_seen,compute_generated_quality,evaluate_quality,evaluate_quality_omni,selective_sample,eval_zs_gzsl,visualize_attention,eval_zs_gzsl_k

# Load data

In [4]:
parser = argparse.ArgumentParser()
parser.add_argument('--critic_iter', type=int, default=5, help='critic iteration, following WGAN-GP')
parser.add_argument('--lambda1', type=float, default=10, help='gradient penalty regularizer, following WGAN-GP')
parser.add_argument('--nz', type=int, default=312, help='size of the latent z vector')
parser.add_argument('--normalize_V', type=bool, default=True, help='normalize_V')
parser.add_argument('--gen_hidden', type=int, default=1024, help='gen_hidden')
opt = parser.parse_known_args()[0] #omit unknown arguments
print(opt)

Namespace(critic_iter=5, gen_hidden=1024, lambda1=10, normalize_V=True, nz=312)


In [5]:
save_path  = save_NFS+'results/Release_AWA_Composer_Normalize_V/'

In [6]:
#%%
idx_GPU = 2
device = torch.device("cuda:{}".format(idx_GPU) if torch.cuda.is_available() else "cpu")
#%%
dataloader = AWA2DataLoader(NFS_path_AoA,device)

/home/project_amadeus/mnt/raptor/hbdat/Attention_over_attention/
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
AWA2
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Balance dataloader
_____
/home/project_amadeus/mnt/raptor/hbdat/Attention_over_attention/data/AWA2/feature_map_ResNet_101_AWA2.hdf5
Expert Attr
threshold at zero attribute with negative value
Finish loading data in  165.109753


# Pretrain DAZLE

In [7]:
#%%
seed = 214
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

print('Randomize seed {}'.format(seed))
#%%

batch_size = 50
nepoches = 20
niters = dataloader.ntrain * nepoches//batch_size
dim_f = 2048
dim_v = 300
init_w2v_att = dataloader.w2v_att
att = dataloader.att#dataloader.normalize_att#
normalize_att = dataloader.normalize_att
#assert (att.min().item() == 0 and att.max().item() == 1)

trainable_w2v = True
lambda_1 = 0.0
lambda_2 = 0.0
lambda_3 = 0.0
bias = 0
prob_prune = 0
uniform_att_1 = False
uniform_att_2 = True

seenclass = dataloader.seenclasses
unseenclass = dataloader.unseenclasses
desired_mass = 1#unseenclass.size(0)/(seenclass.size(0)+unseenclass.size(0))
report_interval = niters//nepoches#10000//batch_size#

model = DAZLE(dim_f,dim_v,init_w2v_att,att,normalize_att,
            seenclass,unseenclass,
            lambda_1,lambda_2,lambda_3,
            device,
            trainable_w2v,normalize_V=opt.normalize_V,normalize_F=True,is_conservative=False,
            uniform_att_1=uniform_att_1,uniform_att_2=uniform_att_2,
            prob_prune=prob_prune,desired_mass=desired_mass, is_conv=False,
            is_bias=False)


Randomize seed 214


/home/project_amadeus/home/hbdat/[RELEASE]_CompositionalZSL/core/DAZLE.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.init_w2v_att = F.normalize(torch.tensor(init_w2v_att))
/home/project_amadeus/home/hbdat/[RELEASE]_CompositionalZSL/core/DAZLE.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.att = F.normalize(torch.tensor(att)).to(device)


------------------------------
Configuration
loss_type CE
normalize V
normalize F
Init word2vec
Linear model
loss_att BCEWithLogitsLoss()
Bilinear attention module
******************************
Measure w2v deviation
new Laplacian smoothing with desire mass 1 4
Compute Pruning loss 0
Second layer attenion conditioned on image features
------------------------------
No sigmoid on attr score


In [8]:
#%%
lr = 0.0001
weight_decay = 0.0001#0.000#0.#
momentum = 0.#0.#
optimizer_m  = optim.RMSprop( model.parameters() ,lr=lr,weight_decay=weight_decay, momentum=momentum)

#%%
print('-'*30)
print('learing rate {}'.format(lr))
print('trainable V {}'.format(trainable_w2v))
print('lambda_1 {}'.format(lambda_1))
print('lambda_2 {}'.format(lambda_2))
print('lambda_3 {}'.format(lambda_3))
print('optimized seen only')
print('optimizer: RMSProp with momentum = {} and weight_decay = {}'.format(momentum,weight_decay))
print('-'*30)

------------------------------
learing rate 0.0001
trainable V True
lambda_1 0.0
lambda_2 0.0
lambda_3 0.0
optimized seen only
optimizer: RMSProp with momentum = 0.0 and weight_decay = 0.0001
------------------------------


In [9]:
save_dir_phase_1 = save_path+'phase_1_checkpoint/'
run_phase_1 = False
if not os.path.isdir(save_dir_phase_1):
    run_phase_1 = True
    os.mkdir(save_path)

In [10]:
if run_phase_1:
    #%% phase 1 convergence of attention model
    for i in range(0,2000):
        model.train()
        optimizer_m.zero_grad()
        batch_label, batch_feature, batch_att = dataloader.next_batch(batch_size)
        out_package = model(batch_feature)

        in_package = out_package
        in_package['batch_label'] = batch_label
        
        out_package=model.compute_loss(in_package)
        loss,loss_CE,loss_w2v,loss_prune,loss_pmp = out_package['loss'],out_package['loss_CE'],out_package['loss_w2v'],out_package['loss_prune'],out_package['loss_pmp']
        entropy = out_package['entropy']
        entropy_A_p = out_package['entropy_A_p']
        loss.backward()
        optimizer_m.step()
        if i%100==0:
            print('-'*30)
            acc_seen, acc_novel, H, acc_zs = eval_zs_gzsl(dataloader,model,device,bias_seen=-bias,bias_unseen=bias)
            print('iter {} loss {} loss_CE {} loss_w2v {} loss_prune: {} loss_pmp: {}'.format(i,loss.item(),loss_CE.item(),loss_w2v.item(),loss_prune.item(),loss_pmp.item()))
            print('entropy {} entropy_A_p {}'.format(entropy,entropy_A_p))
            print('acc_seen {} acc_novel {} H {}'.format(acc_seen, acc_novel, H))  
            print('acc_zs {}'.format(acc_zs))
    os.mkdir(save_dir_phase_1)
    torch.save(model.state_dict(), save_dir_phase_1+'model_phase_1')
else:
    print('-'*30)
    model.load_state_dict(torch.load(save_dir_phase_1+'model_phase_1'))
    print("Load checkpoint")
    acc_seen, acc_novel, H, acc_zs = eval_zs_gzsl(dataloader,model,device,bias_seen=-bias,bias_unseen=bias)
    print('acc_seen {} acc_novel {} H {}'.format(acc_seen, acc_novel, H))
    print('acc_zs {}'.format(acc_zs))

------------------------------
Load checkpoint
bias_seen 0 bias_unseen 0
acc_seen 0.88995361328125 acc_novel 0.2082993984222412 H 0.3375848739675165
acc_zs 0.6440294981002808


# Dense Feature Composition

In [11]:
del model
del optimizer_m
model = DAZLE(dim_f,dim_v,init_w2v_att,att,normalize_att,
            seenclass,unseenclass,
            lambda_1,lambda_2,lambda_3,
            device,
            trainable_w2v,normalize_V=opt.normalize_V,normalize_F=True,is_conservative=True,
            uniform_att_1=uniform_att_1,uniform_att_2=uniform_att_2,
            prob_prune=prob_prune,desired_mass=desired_mass, is_conv=False,
            is_bias=False,margin=1)

print('-'*30)
model.load_state_dict(torch.load(save_dir_phase_1+'model_phase_1'))
print("Load checkpoint")
acc_seen, acc_novel, H, acc_zs = eval_zs_gzsl(dataloader,model,device,bias_seen=-bias,bias_unseen=bias)
print('acc_seen {} acc_novel {} H {}'.format(acc_seen, acc_novel, H))

lr = 0.0001
weight_decay = 0.0001#0.000#0.#
momentum = 0.#0.#
optimizer_m  = optim.RMSprop( model.parameters() ,lr=lr,weight_decay=weight_decay, momentum=momentum)

------------------------------
Configuration
loss_type CE
normalize V
normalize F
training to exclude unseen class [seen upperbound]
Init word2vec
Linear model
loss_att BCEWithLogitsLoss()
Bilinear attention module
******************************
Measure w2v deviation
new Laplacian smoothing with desire mass 1 4
Compute Pruning loss 0
Second layer attenion conditioned on image features
------------------------------
No sigmoid on attr score
------------------------------
Load checkpoint
bias_seen 0 bias_unseen 0
acc_seen 0.88995361328125 acc_novel 0.2082993984222412 H 0.3375848739675165


In [12]:
from core.Composer_AttEmb import Composer_AttEmb

pGenerator = Composer_AttEmb(device=device,base_model = model, k=5,n_comp=50,T=10)

Attribute Embed
Inherit V
Repurpose W_1
Inherit W_1
Using OMP
nearest neighbour k 5
T 10
n_comp 50


### Remark
To prevent seen class bias, we add a fix margin of size 1 to unseen class scores following the DAZLE work.
Learning this margin as a function of input could further improve performances.

In [14]:
from core.Margin import Margin

margin_model = Margin(base_model=model,second_order=True,activate=True)
print("No hidden layer")

lr_p = 0.001
weight_decay_p = 0.01#1#
momentum_p = 0.0#0.#

optimizer_p_m  = optim.RMSprop( margin_model.parameters() ,lr=lr_p,weight_decay=weight_decay_p, momentum=momentum_p)

margin_model.linear1.weight.data  *= 0
margin_model.linear1.bias.data  *= 0
margin_model.linear1.bias.data  += 1

print(margin_model.linear1.weight)
print(margin_model.linear1.bias)

In [15]:
margin_model.linear1.weight.requires_grad = False
margin_model.linear1.bias.requires_grad = False
model.W_1.requires_grad = True
model.V.requires_grad = True
model.W_2.requires_grad = False
model.W_3.requires_grad = False

In [16]:
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

best_performance = [0,0,0,0]

lamb = 1#0.7

tic = time.time()
for i in range(0,1000):
    
    batch_label, batch_feature, batch_att = dataloader.next_batch(batch_size)

    Hs_seen = model.extract_attention(batch_feature)['Hs']#.detach()

    ### random mini-batch from generator ###
    unseen_labels = model.unseenclass[np.random.randint(model.unseenclass.size(0),size=batch_label.size(0))]#model.unseenclass#
    att_unseen = model.att[unseen_labels]
    Hs_unseen = pGenerator.compose(Hs=Hs_seen,labels_s=batch_label,labels_t=unseen_labels)
    
    if Hs_unseen.size(0) == 0:
        continue

    ## seen reinforcement
    optimizer_m.zero_grad()
    optimizer_p_m.zero_grad()
    
    
    out_package_seen = margin_model.compute_attribute_embed(Hs_seen)

    in_package_seen = out_package_seen
    in_package_seen['batch_label'] = batch_label
    
    loss_CE_seen = margin_model.compute_aug_cross_entropy(in_package_seen,is_conservative=True)

    out_package_unseen = margin_model.compute_attribute_embed(Hs_unseen)

    in_package_unseen = out_package_unseen
    in_package_unseen['batch_label'] = unseen_labels
    
    loss_pmp = model.compute_loss_Laplace(in_package_unseen)
    
    loss_CE_unseen = margin_model.compute_aug_cross_entropy(in_package_unseen)
    
    if lamb == -1:
        lamb = loss_CE_seen.item()/loss_CE_unseen.item()
        print("lamb:",lamb)
    
    loss_s_u =  loss_CE_seen +  lamb*loss_CE_unseen
    
    loss_s_u.backward()
    optimizer_p_m.step()
    optimizer_m.step()
    
    if i%50==0:
        print('-'*30)
        print(i)
        bias = 0
        acc_seen, acc_novel, H, acc_zs = eval_zs_gzsl(dataloader,margin_model,device,bias_seen=-bias,bias_unseen=bias)
        
        if H > best_performance[2]:
            best_performance = [acc_seen, acc_novel, H, acc_zs]
        stats_package = {'iter':i, 'loss_s_u':loss_s_u.item(),
                         'acc_seen':best_performance[0], 'acc_novel':best_performance[1], 'H':best_performance[2], 'acc_zs':best_performance[3]}
        
        print(stats_package)
        
        
        tic = time.time()

------------------------------------------------------------
Optimize critic
------------------------------------------------------------
------------------------------
0
bias_seen 0 bias_unseen 0


/home/project_amadeus/home/hbdat/[RELEASE]_CompositionalZSL/core/Composer_AttEmb.py:169: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  categorical_dis = torch.distributions.categorical.Categorical(probs= torch.tensor(multinomial_prob))


{'iter': 0, 'loss_s_u': 2.1748783588409424, 'acc_seen': 0.7501376867294312, 'acc_novel': 0.587559700012207, 'H': 0.6589691787560048, 'acc_zs': 0.6618930697441101}
------------------------------
50
bias_seen 0 bias_unseen 0
{'iter': 50, 'loss_s_u': 1.781742811203003, 'acc_seen': 0.7449354529380798, 'acc_novel': 0.6071327328681946, 'H': 0.6690116698263989, 'acc_zs': 0.6966259479522705}
------------------------------
100
bias_seen 0 bias_unseen 0
{'iter': 100, 'loss_s_u': 1.8810646533966064, 'acc_seen': 0.749400794506073, 'acc_novel': 0.6296268701553345, 'H': 0.6843124163903286, 'acc_zs': 0.7190256118774414}
------------------------------
150
bias_seen 0 bias_unseen 0
{'iter': 150, 'loss_s_u': 1.9518096446990967, 'acc_seen': 0.749400794506073, 'acc_novel': 0.6296268701553345, 'H': 0.6843124163903286, 'acc_zs': 0.7190256118774414}
------------------------------
200
bias_seen 0 bias_unseen 0
{'iter': 200, 'loss_s_u': 1.826515555381775, 'acc_seen': 0.749400794506073, 'acc_novel': 0.629626870